In [ ]:
import pandas as pd
import numpy as np
from scipy.fft import fft, ifft
from scipy.spatial.distance import euclidean
import matplotlib.pyplot as plt

# Configuración de datos de entrada
# Cargar los datos
nombres_csv = {1,2,3,4,468,479,4003,4151,4160,4173}
for nombre_csv in nombres_csv:
    df= pd.read_csv(f'Data2_actividad/{nombre_csv}_act.csv')
    df = df.copy()

    # Convertir 'Fecha' y 'Hora' a datetime e indexar el DataFrame
    df['datetime'] = pd.to_datetime(df['Fecha'] + ' ' + df['Hora'].astype(str) + ':00:00')
    df = df.set_index('datetime')
    df = df.sort_index()

    # Función para calcular el ciclo circadiano utilizando la Transformada de Fourier
    def extract_circadian_cycle(data):
        fft_result = fft(data)
        n = len(data)
        freq = np.fft.fftfreq(n, d=1/24)
        
        # Filtrar para mantener solo frecuencias cercanas a 1 ciclo diario
        fft_filtered = fft_result.copy()
        fft_filtered[(freq < 0.8) | (freq > 1.2)] = 0
        
        # Reconstrucción de la señal para obtener el ciclo circadiano
        return np.real(ifft(fft_filtered))

    # Función para procesar una columna de actividad y calcular niveles de estrés
    def process_activity(df, activity_column):
        daily_cycles = {}
        
        # Extraer ciclo circadiano para cada día y almacenarlo
        for date, group in df.groupby(df.index.date):
            if len(group) == 24:  # Verificar 24 horas de datos
                daily_cycles[date] = extract_circadian_cycle(group[activity_column].values)

        # Calcular el ciclo circadiano promedio
        average_cycle = np.mean(list(daily_cycles.values()), axis=0)

        # Calcular distancias entre el ciclo promedio y los ciclos diarios
        hourly_distances = []
        for date, cycle in daily_cycles.items():
            for hour in range(24):
                hourly_distances.append({
                    'Fecha': date,
                    'Hora': hour,
                    'distancia': euclidean(average_cycle, cycle)
                })

        distance_df = pd.DataFrame(hourly_distances)

        # Calcular umbrales para clasificar niveles de estrés
        min_distance = distance_df['distancia'].min()
        max_distance = distance_df['distancia'].max()
        threshold1 = min_distance + (max_distance - min_distance) / 2
        threshold2 = threshold1 + (max_distance - threshold1) / 2
        threshold3 = max_distance

        # Clasificación de niveles de estrés basada en los umbrales
        def get_stress_level(distance):
            if distance <= threshold1:
                return 'normal'
            elif distance <= threshold2:
                return 'alerta'
            else:
                return 'peligro'

        # Aplicar la función de nivel de estrés a cada distancia
        distance_df['stress_level'] = distance_df['distancia'].apply(get_stress_level)
        
        return distance_df[['Fecha', 'Hora', 'stress_level']]

    # Procesar ambas columnas de actividad y calcular niveles de estrés
    stress_levels_1 = process_activity(df, 'activity_level').rename(columns={'stress_level': 'stress_level_1'})
    stress_levels_2 = process_activity(df, 'activity_level_2').rename(columns={'stress_level': 'stress_level_2'})

    # Fusionar niveles de estrés con el DataFrame original
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    stress_levels_1['Fecha'] = pd.to_datetime(stress_levels_1['Fecha'])
    stress_levels_2['Fecha'] = pd.to_datetime(stress_levels_2['Fecha'])
    df['Hora'] = df['Hora'].astype(int)
    stress_levels_1['Hora'] = stress_levels_1['Hora'].astype(int)
    stress_levels_2['Hora'] = stress_levels_2['Hora'].astype(int)

    # Fusionar todos los DataFrames en df_final
    df_final = df.merge(stress_levels_1, on=['Fecha', 'Hora'], how='left')
    df_final = df_final.merge(stress_levels_2, on=['Fecha', 'Hora'], how='left')

    df_final.to_csv(f'Datos_train/{nombre_csv}_p.csv', index=False) 
    df_final.to_csv(f'Datos_test/{nombre_csv}_p.csv', index=False)   
